<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/03_%E3%83%AC%E3%82%B8%E3%82%B9%E3%83%88%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# レジストレーション

## コードをダウンロード

In [ ]:
!git clone --recursive https://github.com/3d-point-cloud-processing/3dpcp_book_codes

使用するデータをダウンロード

In [ ]:
!wget http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz
!tar xvfz ./bunny.tar.gz

## ライブラリをダウンロード

In [ ]:
!pip install open3d==0.16

## 最近傍探索

### 単純な全探索

In [ ]:
import open3d as o3d
import numpy as np
import copy


# point cloud as sin function
X_x = np.arange(-np.pi,np.pi, 0.1)
X_y = np.sin(X_x)*3
X_z = np.zeros(X_x.shape)
X = np.vstack([X_x, X_y, X_z]).T

# point p
p = np.array([1.0,0.0,0.0])

# open3d point cloud of X
pcd_X = o3d.geometry.PointCloud()
pcd_X.points = o3d.utility.Vector3dVector(X)
pcd_X.paint_uniform_color([0.5,0.5,0.5])

# open3d point cloud of p
pcd_p = o3d.geometry.PointCloud()
pcd_p.points = o3d.utility.Vector3dVector([p])
pcd_p.paint_uniform_color([0.0,0.0,1.0])

# Visualization
o3d.visualization.draw_plotly(
  [pcd_X,pcd_p],
  width=800,
  height=800
)



In [ ]:
def dist( p, X ):
    dists = np.linalg.norm(p-X,axis=1, ord=2)  # 距離の計算
    min_dist = min(dists)
    min_idx = np.argmin(dists)
    
    return min_dist, min_idx

min_dist, min_idx = dist(p,X)
np.asarray(pcd_X.colors)[min_idx] = [0.0,1.0,0.0]
print("distance:{}, idx:{}".format(min_dist, min_idx))
# o3d.visualization.draw_geometries([mesh,pcd_X,pcd_p])
o3d.visualization.draw_plotly(
  [pcd_X,pcd_p],
  width=800,
  height=800
)


### KD-TREE
kd-treeによる最近傍探索を行います。

比較として、全探索を行った場合の処理時間を計測してみます。


In [ ]:
import open3d as o3d
import numpy as np
import copy
import time 

pcd = o3d.io.read_point_cloud("/content/bunny/data/bun000.ply")
pcd.paint_uniform_color([0.5, 0.5, 0.5])
distslist = []
# 全探索による近傍探索開始
start = time.time()
for pt in pcd.points:
  dists = np.linalg.norm(pt - pcd.points[10000],axis=0, ord=2)
  distslist.append(dists)

distslist = np.array(distslist)
distslist_index = distslist.argsort()
np.asarray(pcd.colors)[distslist_index[1:200], :] = [0 , 1 , 1]
# 全探索による近傍探索終了
end = time.time()
print(end -start)
o3d.visualization.draw_plotly(
  [pcd],
  width=800,
  height=800
)


KD-treeを使用したらどれくらい処理が早くなるかを確認します。


In [ ]:
import open3d as o3d
import numpy as np
import copy

pcd = o3d.io.read_point_cloud("/content/bunny/data/bun000.ply")
pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

query = 10000
pcd.colors[query] = [1, 0, 0]
# kd-treeによる近傍探索
start = time.time()
[k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[query], 200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]
# kd-treeによる近傍探索
end = time.time()
print(end -start)

o3d.visualization.draw_geometries([pcd], width=600, height=400)

# query = 20000
# pcd.colors[query] = [1, 0, 0]
# [k, idx, d] = pcd_tree.search_radius_vector_3d(pcd.points[query], 0.01)
# np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]
# o3d.visualization.draw_geometries([pcd], width=600, height=400)

# query = 5000
# pcd.colors[query] = [1 , 0 , 0]
# [k ,idx ,d ] = pcd_tree.search_hybrid_vector_3d( pcd.points[query],
#                                                         radius =0.01,
#                                                         max_nn =200 )
np.asarray(pcd.colors)[idx[1:], :] = [0 , 1 , 1]
o3d.visualization.draw_plotly(
  [pcd],
  width=800,
  height=800
)


## ICPアルゴリズム

In [ ]:
import open3d as o3d
import numpy as np
import copy


pcd1 = o3d.io.read_point_cloud("/content/bunny/data/bun000.ply")
pcd2 = o3d.io.read_point_cloud("/content/bunny/data/bun045.ply")


pcd_s = pcd1.voxel_down_sample(voxel_size=0.003)
pcd_t = pcd2.voxel_down_sample(voxel_size=0.003)

# 初期状態の表示
pcd_s.paint_uniform_color([0.0, 1.0, 0.0])
pcd_t.paint_uniform_color([0.0, 0.0, 1.0])
o3d.visualization.draw_geometries([pcd_s,pcd_t])

# ICPによる位置合わせ
threshold = 0.05
trans_init = np.identity(4)
obj_func = o3d.pipelines.registration.TransformationEstimationPointToPoint()
result = o3d.pipelines.registration.registration_icp( pcd_s, pcd_t,
                                                      threshold,
                                                      trans_init,
                                                      obj_func
                                                    )

trans_reg = result.transformation
print(trans_reg) 

# 得られた変換行列を点群に適用
pcd_reg = copy.deepcopy(pcd_s).transform(trans_reg)
pcd_reg.paint_uniform_color([1.0, 0.0, 0.0])


o3d.visualization.draw_plotly(
  [pcd_reg,pcd_t],
  width=800,
  height=800
)